In [12]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.authorities.topic import Topic, Authority
from datetime import datetime
from api.src.function.authorities.topic import template
from enum import Enum
from pydantic import BaseModel, validator

In [17]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')

In [13]:
obj = {
  "value":
    "Metodo",
  "lang": "pt",
  "type": "Topic"
}
request = Authority(**obj)
request

Authority(value='Metodo', lang='pt', type='Topic')

In [18]:
label = """PREFIX topic: <https://bibliokeia.com/authorities/topic/>
                PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                
                WITH topic:{id}
                DELETE {{ topic:{id} madsrdf:authoritativeLabel ?o  }}
                INSERT {{ topic:{id} madsrdf:authoritativeLabel '{value}'@{lang} }}
                WHERE {{ topic:{id} madsrdf:authoritativeLabel ?o }}"""

d = request.dict()
d['id'] = 'sh85017405'
upLabel = label.format(**d)
responde = fuseki_update.run_sparql(upLabel)
print(responde.convert())

{'statusCode': 200, 'message': 'Update succeeded'}


In [19]:
d

{'value': 'Metodo', 'lang': 'pt', 'type': 'Topic', 'id': 'sh85017405'}

In [ ]:
doc = {
    'id': 'sh85017405',
    'label': {"set": 'Metodo'},
    'lang': {"set": 'pt'}
}

SOLR

In [5]:
obj = {
  "identifier": "sh85017405",
  "label": "Methodology",
  "lang": "en",
  "hasReciprocalAuthority": ["http://id.loc.gov/authorities/subjects/sh85113021"],
  "hasBroaderAuthority" : ["http://id.loc.gov/authorities/subjects/sh85017404"],
 "hasExactExternalAuthority": ["http://id.loc.gov/authorities/classification/BD240-BD241"],
 "hasCloseExternalAuthority" : [
    "http://d-nb.info/gnd/4061488-8",
        "http://data.bnf.fr/ark:/12148/cb12345685v",
        "http://id.worldcat.org/fast/839960",
        "http://www.wikidata.org/entity/Q130933",
        "http://www.yso.fi/onto/yso/p21203" 
  ],
  "hasNarrowerAuthority": [
    "http://id.loc.gov/authorities/subjects/sh00005679",
    "http://id.loc.gov/authorities/subjects/sh00005984",
    "http://id.loc.gov/authorities/subjects/sh2001008679"
  ],
  "hasVariant": [
    { "value": "Northern eagle owl",
    "type": "Topic",
    "lang": "en" },
    { "value": "Great eagle owl",
    "type": "Topic",
    "lang": "en" },
    { "value": ["Philosophy","Methodology"],
    "type": "ComplexSubject",
    "lang": "en" }
  ]
    }

request = Topic(**obj)
d = request.dict()
d

{'recordChangeDate': '2023-04-02T19:03:19',
 'identifier': 'sh85017405',
 'label': 'Methodology',
 'lang': 'en',
 'hasReciprocalAuthority': ['http://id.loc.gov/authorities/subjects/sh85113021'],
 'hasBroaderAuthority': ['http://id.loc.gov/authorities/subjects/sh85017404'],
 'hasNarrowerAuthority': ['http://id.loc.gov/authorities/subjects/sh00005679',
  'http://id.loc.gov/authorities/subjects/sh00005984',
  'http://id.loc.gov/authorities/subjects/sh2001008679'],
 'hasCloseExternalAuthority': ['http://d-nb.info/gnd/4061488-8',
  'http://data.bnf.fr/ark:/12148/cb12345685v',
  'http://id.worldcat.org/fast/839960',
  'http://www.wikidata.org/entity/Q130933',
  'http://www.yso.fi/onto/yso/p21203'],
 'hasExactExternalAuthority': ['http://id.loc.gov/authorities/classification/BD240-BD241'],
 'hasVariant': [{'value': 'Northern eagle owl', 'lang': 'en', 'type': 'Topic'},
  {'value': 'Great eagle owl', 'lang': 'en', 'type': 'Topic'},
  {'value': ['Philosophy', 'Methodology'],
   'lang': 'en',
   

In [8]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')

prefix = """PREFIX topic: <https://bibliokeia.com/authorities/topic/>
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""
bkID = "sh85017405"
up = prefix+"WITH topic:"+bkID+"""
            DELETE { topic:sh85017405 madsrdf:authoritativeLabel ?o  }
            INSERT { topic:sh85017405 madsrdf:authoritativeLabel 'Metodologia'@pt }
            WHERE { topic:sh85017405 madsrdf:authoritativeLabel ?o }"""

response = fuseki_update.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [10]:
prefix = """PREFIX topic: <https://bibliokeia.com/authorities/topic/>
            PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>"""
bkID = "sh85017405"
up = prefix+"WITH topic:"+bkID+"""
            DELETE { topic:sh85017405 madsrdf:elementList ?elementList .
        ?elementList rdf:rest rdf:nil .
        ?elementList rdf:first ?element .
        ?element rdf:type madsrdf:TopicElement .
        ?element madsrdf:elementValue ?value  }
            INSERT { topic:sh85017405 madsrdf:elementList ?elementList .
        ?elementList rdf:rest rdf:nil .
        ?elementList rdf:first ?element .
        ?element rdf:type madsrdf:TopicElement .
        ?element madsrdf:elementValue 'Metodologia'@pt }
            WHERE { topic:sh85017405 madsrdf:elementList ?elementList .
        ?elementList rdf:rest rdf:nil .
        ?elementList rdf:first ?element .
        ?element rdf:type madsrdf:TopicElement .
        ?element madsrdf:elementValue ?value }"""

response = fuseki_update.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [16]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)

In [ ]:
doc = {
    'id': request.identifier,
    'uri': f'https://bibliokeia.com/authorities/topic/{request.identifier}'
}

for k, v in request:
    if v:
        if k == 'hasVariant':
            variant = [i['value'] for i in v]
            doc['variant'] = variant
        else:
            print(v)
doc

In [15]:
for i in request.hasVariant:
    variant = ["--".join(i.value) if i.type == 'ComplexSubject' else i.value for i in request.hasVariant]
    print(variant)

['Northern eagle owl', 'Great eagle owl', 'Philosophy--Methodology']
['Northern eagle owl', 'Great eagle owl', 'Philosophy--Methodology']
['Northern eagle owl', 'Great eagle owl', 'Philosophy--Methodology']


In [7]:
graph = template.format(**request.dict())
broader = "madsrdf:hasBroaderAuthority {} ; \n"
closeExternal = "madsrdf:hasCloseExternalAuthority {} ; \n"
exactExternal = "madsrdf:hasExactExternalAuthority {} ; \n"
hasVariant = "madsrdf:hasVariant {} ; \n"

variant = """[ a madsrdf:Topic,
                madsrdf:Variant ;
            madsrdf:elementList ( [ a madsrdf:TopicElement ;
                        madsrdf:elementValue "{value}"@{lang} ] ) ;
            madsrdf:variantLabel "{value}"@{lang} ]"""

variants = list()
for i in request.hasVariant:
    v = variant.format(**i)
    variants.append(v)

x = f"{graph} \
{ broader.format(', '.join([ f'<{i}>' for i in request.hasBroaderAuthority])) if request.hasBroaderAuthority else '' } \
{ closeExternal.format(', '.join([ f'<{i}>' for i in request.hasCloseExternalAuthority])) if request.hasCloseExternalAuthority else '' } \
        { exactExternal.format(', '.join([ f'<{i}>' for i in request.hasExactExternalAuthority])) if request.hasExactExternalAuthority else '' } \
    { hasVariant.format(', '.join(variants)) if request.hasVariant else '' } }} }}"
print(x)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
PREFIX topic: <https://bibliokeia.com/authorities/topic/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

INSERT DATA {
    GRAPH topic:sh2018002121 {
        topic:sh2018002121 a madsrdf:Authority, madsrdf:Topic ;
        madsrdf:adminMetadata [ a ri:RecordInfo ; 
        ri:recordChangeDate "2023-04-02T09:56:28"^^xsd:dateTime ;
        ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
        ri:recordStatus "new"^^xsd:string
        ] ; 
        madsrdf:authoritativeLabel "Dark Web"@en ;
        madsrdf:elementList ( [ a madsrdf:TopicElement ;
                madsrdf:elementValue "Dark Web"@en ] ) ;    
 madsrdf:hasBroaderAuthority <http://id.loc.gov/authorities/subjects/sh2008009697> ; 
 madsrdf:hasCloseExternalAuthority <http://id.worldcat.org

In [8]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')

In [18]:
response = fuseki_update.run_sparql(x)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [6]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = fuseki_update.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [23]:
g.parse('https://www.loc.gov/standards/mads/rdf/v1/examples/sh85017405.ttl', format='ttl')

<Graph identifier=N5177ed9befcc4810871e74518a3d960a (<class 'rdflib.graph.Graph'>)>

In [9]:
g = Graph()
g.parse('http://id.loc.gov/authorities/subjects/sh85084414.madsrdf.nt')

HTTPError: HTTP Error 503: Service Unavailable

In [31]:
g.serialize('topic.ttl', format='ttl')

<Graph identifier=Nde9974b3e3f24e3db4ab813f596d48e9 (<class 'rdflib.graph.Graph'>)>

POST TOPIC

In [32]:
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
PREFIX subjects: <https://bibliokeia.com/authorities/subjects/> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> """

In [11]:
topic ="""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
PREFIX bk: <https://bibliokeia.com/authorities/subjects/> 

INSERT DATA {
    GRAPH bk:sh85017405 {
    bk:sh85017405
    madsrdf:adminMetadata [
        ri:recordChangeDate "1986-02-11T00:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime> ;
        ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/dlc> ;
        ri:recordStatus "new"^^<http://www.w3.org/2001/XMLSchema#string> ;
        a ri:RecordInfo
    ] ;
    madsrdf:authoritativeLabel "Bubo bubo"@en ;
    madsrdf:classification "QL696.S83" ;
    madsrdf:elementList ([
            madsrdf:elementValue "Bubo bubo" ;
            a madsrdf:TopicElement
        ]
    ) ;
    madsrdf:hasBroaderAuthority <http://id.loc.gov/authorities/sh85017404> ;
    madsrdf:hasEarlierEstablishedForm <http://id.loc.gov/authorities/sh85017405> ;
    madsrdf:hasSource [
        madsrdf:citation-note "(for Genus bubo)"@en ;
        madsrdf:citation-source "Web. 3" ;
        madsrdf:citation-status "found" ;
        a madsrdf:Source
    ], [
        madsrdf:citation-note "(Bubo bubo)"@en ;
        madsrdf:citation-source "Howard birds world" ;
        madsrdf:citation-status "found" ;
        a madsrdf:Source
    ] ;
    madsrdf:hasVariant [
        madsrdf:elementList ([
                madsrdf:elementValue "Eagle owl" ;
                a madsrdf:TopicElement
            ]
        ) ;
        madsrdf:variantLabel "Eagle owl"@en ;
        a madsrdf:Topic, madsrdf:Variant
    ], [
        madsrdf:elementList ([
                madsrdf:elementValue "Great eagle owl" ;
                a madsrdf:TopicElement
            ]
        ) ;
        madsrdf:variantLabel "Great eagle owl"@en ;
        a madsrdf:Topic, madsrdf:Variant
    ], [
        madsrdf:elementList ([
                madsrdf:elementValue "Northern eagle owl" ;
                a madsrdf:TopicElement
            ]
        ) ;
        madsrdf:variantLabel "Northern eagle owl"@en ;
        a madsrdf:Topic, madsrdf:Variant
    ] ;
    madsrdf:isMemberOfMADSCollection <http://id.loc.gov/authorities/collection_AuthorizedHeadings>, <http://id.loc.gov/authorities/collection_LCSH_General>, <http://id.loc.gov/authorities/collection_SubdivideGeographicalIndirect> ;
    madsrdf:isMemberOfMADSScheme <http://id.loc.gov/authorities/lcsh> ;
    a madsrdf:Authority, madsrdf:Topic ;
    owl:sameAs <http://id.loc.gov/authorities/sh85017405#concept>, <info:lc/authorities/sh85017405> .
    
    }
}"""

In [12]:
response = fuseki_update.run_sparql(topic)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [13]:
uri = f'http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/subjects/sh85017405'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('subjects.nt')

<Graph identifier=Nccdedf49a77545199c0cfbcd222f62de (<class 'rdflib.graph.Graph'>)>

Edit Authority
ENDPOINT: 
@put
mads/authority 

REQUEST:
{ 'oldValue' : string,
    'oldLang': string,
    'newValue': string,
    'newLang: string
}

In [16]:
subject_id = "sh85017405"

In [23]:


upLabel = prefix+"WITH subjects:"+subject_id+"""
                DELETE { subjects:"""+subject_id+""" madsrdf:authoritativeLabel ?o }
                INSERT { subjects:"""+subject_id+""" madsrdf:authoritativeLabel "Bubo"@en }
                WHERE { subjects:"""+subject_id+""" madsrdf:authoritativeLabel ?o }"""

response = fuseki_update.run_sparql(upLabel)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [22]:
upValue = prefix+"WITH subjects:"+subject_id+"""
            DELETE { subjects:sh85017405 madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element .
    ?element madsrdf:elementValue "Bubo bubo" }
            INSERT { subjects:sh85017405 madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element .
    ?element madsrdf:elementValue "Bubo" }
            WHERE { subjects:sh85017405 madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element .
    ?element madsrdf:elementValue "Bubo bubo" }"""

response = fuseki_update.run_sparql(upValue)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

EDIT VARIANT

Acrescenta uma variant: <br>
@post<br>
ENDPOINT:<br>
mads/variant<br>


In [24]:
insert = prefix+"""INSERT DATA
{ GRAPH subjects:"""+subject_id+""" { 
    subjects:"""+subject_id+""" madsrdf:hasVariant [
    rdf:type madsrdf:Topic;
    rdf:type madsrdf:Variant ;
    madsrdf:elementList ( [
    rdf:type madsrdf:TopicElement;
    madsrdf:elementValue 'NOVA VARIANTE'
    ]);
    madsrdf:variantLabel 'NOVA VARIANTE'
    ] 
 } }"""

response = fuseki_update.run_sparql(insert)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Exclui uma variant: <br>
@del<br>
ENDPOINT:<br>
mads/variant<br>

In [28]:
up = prefix+"WITH subjects:"+subject_id+"""

        DELETE {  subjects:sh85017405 madsrdf:hasVariant ?hasVariant .
    ?hasVariant ?p1  "Eagle owl"@en .
    ?hasVariant rdf:type madsrdf:Variant  .
    ?hasVariant rdf:type madsrdf:Topic  .
     ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element . 
    ?elementList rdf:rest rdf:nil . 
    ?element ?p3 "Eagle owl" .
    ?element rdf:type madsrdf:TopicElement 
      }  
      WHERE {
     subjects:sh85017405 madsrdf:hasVariant ?hasVariant .
    ?hasVariant ?p1  "Eagle owl"@en .
    ?hasVariant rdf:type madsrdf:Variant  .
    ?hasVariant rdf:type madsrdf:Topic  .
     ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element . 
    ?elementList rdf:rest rdf:nil . 
    ?element ?p3 "Eagle owl" .
    ?element rdf:type madsrdf:TopicElement 
      
      }           
      """
response = fuseki_update.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Editar uma variant: <br>
@put<br>
ENDPOINT:<br>
mads/variant<br>

In [29]:
up = prefix+"WITH subjects:"+subject_id+"""
        DELETE {  subjects:sh85017405 madsrdf:hasVariant ?hasVariant .
    ?hasVariant ?p1  "Great eagle owl"@en .
    ?hasVariant rdf:type madsrdf:Variant  .
    ?hasVariant rdf:type madsrdf:Topic  .
     ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element . 
    ?elementList rdf:rest rdf:nil . 
    ?element ?p3 "Great eagle owl" .
    ?element rdf:type madsrdf:TopicElement 
      }  
      INSERT {
      subjects:sh85017405 madsrdf:hasVariant ?hasVariant .
    ?hasVariant ?p1  "GRANDE AGUIA"@pt .
    ?hasVariant rdf:type madsrdf:Variant  .
    ?hasVariant rdf:type madsrdf:Topic  .
     ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element . 
    ?elementList rdf:rest rdf:nil . 
    ?element ?p3 "GRANDE AGUIA" .
    ?element rdf:type madsrdf:TopicElement 
      }
      WHERE {
     subjects:sh85017405 madsrdf:hasVariant ?hasVariant .
    ?hasVariant ?p1  "Great eagle owl"@en .
    ?hasVariant rdf:type madsrdf:Variant  .
    ?hasVariant rdf:type madsrdf:Topic  .
     ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element . 
    ?elementList rdf:rest rdf:nil . 
    ?element ?p3 "Great eagle owl" .
    ?element rdf:type madsrdf:TopicElement 
      }           
      """
response = fuseki_update.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}



Adiciona um termo restrito: <br>
@post<br>
ENDPOINT:<br>
mads/hasBroaderAuthority<br>


In [30]:
insert = prefix+"""INSERT DATA
{ GRAPH subjects:"""+subject_id+""" { 
    subjects:"""+subject_id+""" madsrdf:hasBroaderAuthority <http://id.loc.gov/authorities/TESTE>
 } }"""

response = fuseki_update.run_sparql(insert)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

DELETE

In [31]:
up = prefix+"WITH subjects:"+subject_id+"""

        DELETE {  subjects:"""+subject_id+""" madsrdf:hasBroaderAuthority <http://id.loc.gov/authorities/TESTE>
      }  
      WHERE {
     subjects:"""+subject_id+""" madsrdf:hasBroaderAuthority <http://id.loc.gov/authorities/TESTE>
      
      }           
      """
response = fuseki_update.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [ ]:
up = prefix+"WITH subjects:"+subject_id+"""
        DELETE {  subjects:sh85017405 madsrdf:adminMetadata ?adminMetadata .
    ?adminMetadata ri:recordChangeDate ?o  
      }  
      INSERT {
      subjects:sh85017405 madsrdf:adminMetadata ?adminMetadata .
    ?adminMetadata ri:recordChangeDate ?o  
      }
      WHERE {
    subjects:sh85017405 madsrdf:adminMetadata ?adminMetadata .
    ?adminMetadata ri:recordChangeDate ?o  
      }           
      """
response = fuseki_update.run_sparql(up)
response.convert()

SCHEMA

In [3]:
uri = "http://localhost:3030/authorities?graph=https:%2F%2Fbibliokeia.com%2Fauthorities%2Ftopic%2Fsh85017405"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('bk-1.ttl')

<Graph identifier=N320154073f3c45bf8ab2e6f4b7be491b (<class 'rdflib.graph.Graph'>)>